# Anonymization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import datetime

Utilizaremos un conjunto de datos sintético de observaciones de signos vitales de pacientes.

In [ ]:
data = pd.read_csv("data/observations.csv",parse_dates=["birth_date", "observation_date"])

Nuestro conjunto de datos contiene una serie de características que podrían categorizarse como datos personales o sensibles.

El nombre y el rut podrían son datos personales que no debiésemos tener acceso, pero son necesarios para la identificación de sujetos dentro del conjunto de datos y para realizar posibles cruces con otras tablas de datos.

Si bien las fechas asociadas al paciente y a las observaciones no nos permiten realizar una detección precisa de la persona a la que pertenecen las instancias de datos, este dato se considera información protegida de salud. No nos interesan las fechas exactas asociadas, pero si nos interesan los delta de tiempo entre observaciones.

Estos dos tipos de datos descritos anteriormente son información protegida, pero de todos modos es información clave para la generación de conclusiones, por lo que no se pueden eliminar, por ende debemos encontrar una solución que nos permita trabajar con estas características sin vulnerar la privacidad del paciente.

In [ ]:
data

Calculamos la edad que tiene el paciente en cada observación.

In [ ]:
data["age"] = (data.observation_date - data.birth_date).dt.days / 365.25
data

Agrupamos por paciente y diagnosticamos a cada paciente con hipertensión.

In [ ]:
hypertension = data.groupby(by=["name"])[["heart_rate","diastolic_blood_pressure","systolic_blood_pressure"]].mean()
hypertension["hypertension"] = hypertension.systolic_blood_pressure > 130
hypertension

Exploramos si existe una correlación entre las observaciones y el género del paciente.

In [ ]:
data.boxplot(column=["heart_rate","diastolic_blood_pressure","systolic_blood_pressure"], by="gender")
plt.tight_layout()
plt.show()

Exploramos si existe una correlación entre la presión arterial y la edad de los pacientes.

In [ ]:
data.plot.scatter(x="age", y="diastolic_blood_pressure")

## Anonimización

Debemos encontrar una forma de codificar la información protegida de los pacientes de una manera que se mantenga la biyección entre la forma bruta y anonimizada de los datos, sin vulnerar la privacidad del paciente y sin que se pueda deidentificar la información. Una de los métodos que cumple todos estos requisitos son las funciones hash.



In [ ]:
rut = "1900686-3"

In [ ]:
hashed_rut = hashlib.md5(
    rut.encode()
).hexdigest()

In [ ]:
hashed_rut

In [ ]:
int(hashed_rut, 16)

In [ ]:
int(hashed_rut[:3], 16)

In [ ]:
def hasher(x,output="hex"):
    encoded_string = x.encode()
    hashed_string = hashlib.md5(encoded_string)
    hashed_hex = hashed_string.hexdigest()
    if output == "int":
        hashed_int = int(hashed_hex[:3], 16)
        return hashed_int
    if output == "hex":
        return hashed_hex

In [ ]:
hasher(rut)

In [ ]:
hasher(rut, output="int")

In [ ]:
anonymized_data = data.copy()

In [ ]:
anonymized_data["name"] = pd.NA
anonymized_data

In [ ]:
anonymized_data["hashed_rut"] = anonymized_data.rut.apply(hasher)
anonymized_data

In [ ]:
anonymized_data["birth_date"] = anonymized_data.apply(lambda x: x["birth_date"] + datetime.timedelta(days=hasher(x["rut"], output="int")), axis=1)
anonymized_data["observation_date"] = anonymized_data.apply(lambda x: x["observation_date"] + datetime.timedelta(days=hasher(x["rut"], output="int")), axis=1)
anonymized_data["age"] = (anonymized_data.observation_date - anonymized_data.birth_date).dt.days / 365.25
anonymized_data

In [ ]:
anonymized_data = anonymized_data.drop("rut",axis=1)

In [ ]:
anonymized_data.groupby(by=["hashed_rut"])[["heart_rate","diastolic_blood_pressure","systolic_blood_pressure"]].mean()

In [ ]:
anonymized_data.boxplot(column=["heart_rate","diastolic_blood_pressure","systolic_blood_pressure"], by="gender")
plt.tight_layout()
plt.show()

In [ ]:
anonymized_data.plot.scatter(x="age", y="diastolic_blood_pressure")